# Capítulo 04: Leitura de Tabelas Delta

Técnicas avançadas para ler e processar tabelas Delta usando DuckDB com Pandas, Polars e Arrow.

## 📦 Setup e Instalação

In [ ]:
%pip install duckdb deltalake pandas polars pyarrow -q

import duckdb
import pandas as pd
import pyarrow as pa

con = duckdb.connect(':memory:')
print(f"✓ DuckDB {duckdb.__version__}")

## 🐼 Leitura para Pandas DataFrame

Conversão de tabelas Delta para Pandas para análise interativa.

In [ ]:
# Ler Delta table para Pandas
df = con.execute("""
    SELECT *
    FROM delta_scan('./delta_tables/sales')
    WHERE order_date >= '2024-01-01'
    LIMIT 1000
""").df()

print(f"✓ {len(df)} registros carregados")
print("\nPrimeiros registros:")
print(df.head())

print("\nEstatísticas descritivas:")
print(df[['amount', 'quantity']].describe())

## 🚀 Leitura para Polars (High Performance)

Conversão para Polars para análises de alto desempenho.

In [ ]:
import polars as pl

# Ler para Polars
df_polars = con.execute("""
    SELECT *
    FROM delta_scan('./delta_tables/sales')
    LIMIT 10000
""").pl()

# Análise agregada com Polars
result = (
    df_polars
    .group_by('customer_id')
    .agg([
        pl.col('amount').sum().alias('total_spent'),
        pl.col('order_id').count().alias('order_count'),
        pl.col('amount').mean().alias('avg_order')
    ])
    .sort('total_spent', descending=True)
    .head(10)
)

print("Top 10 Clientes por Valor Total:")
print(result)

## 📊 Dashboard de Vendas

Função completa para gerar dashboard analítico com métricas de vendas.

In [ ]:
from datetime import datetime, timedelta

delta_path = './delta_tables/sales'
days_back = 90

# Métricas principais
metrics = con.execute(f"""
    SELECT
        COUNT(DISTINCT customer_id) as unique_customers,
        COUNT(*) as total_orders,
        SUM(amount) as total_revenue,
        AVG(amount) as avg_order_value,
        MAX(order_date) as last_order_date
    FROM delta_scan('{delta_path}')
    WHERE order_date >= CURRENT_DATE - INTERVAL '{days_back} days'
""").df()

print(f"=== DASHBOARD ({days_back} dias) ===")
print(f"Clientes Únicos: {metrics['unique_customers'][0]:,}")
print(f"Total Pedidos: {metrics['total_orders'][0]:,}")
print(f"Receita Total: ${metrics['total_revenue'][0]:,.2f}")
print(f"Ticket Médio: ${metrics['avg_order_value'][0]:,.2f}")

# Top 10 clientes
top_customers = con.execute(f"""
    SELECT
        customer_id,
        COUNT(*) as orders,
        SUM(amount) as lifetime_value
    FROM delta_scan('{delta_path}')
    WHERE order_date >= CURRENT_DATE - INTERVAL '{days_back} days'
    GROUP BY customer_id
    ORDER BY lifetime_value DESC
    LIMIT 10
""").df()

print("\nTop 10 Clientes:")
print(top_customers)

# Capitulo 04 Leitura Tabelas Delta

Notebook gerado automaticamente a partir do código fonte python.
